# Introduction to DialogueFactory
## Generating a dialogue

This notebook demonstrates how to generate new dialogues and evaluate them using your own policy. We create a random dialogue in the "easy" environment and run it.

In [ ]:
import os

from dialoguefactory import DialogueGenerator
import dialoguefactory.environments.easy as easy_env

home_directory = os.path.expanduser('~')

error_path = os.path.join(home_directory, 'dialoguefactory_logs', 'error.log')
context_path = os.path.join(home_directory, 'dialoguefactory_logs', 'context.log')
os.makedirs(os.path.dirname(error_path), exist_ok=True)
os.makedirs(os.path.dirname(context_path), exist_ok=True)


easy_world = easy_env.build_world()
generator = DialogueGenerator(easy_world, error_path, context_path)
dialogue = generator.generate_dialogue()

dialogue.run()
print ("The utterances:")
for utter in dialogue.utterances:
    print (utter.to_string())
    
evaluated_dia = dialogue.evaluate_goal()
if evaluated_dia == 1:
    print ("Success")
elif evaluated_dia == 0:
    print ("In progress")
else:
    print ("Failed")


As you can see, the dialogue consists of the user issuing a request to an agent. The agent's goal is to fulfill the user's request.

The dialogue's goal is success because we use an Oracle agent's policy. Now, let us replace it with a dummy policy. 

The policies must have an execute function that returns the next agent's response. The get_goal function returns the agent's goal. We use the get_goal function from the oracle's policy as the dialogue's goal. For the machine learning policy, this function should return None. 


In [ ]:
import dialoguefactory.language.components as lc
import dialoguefactory.language.sentences as sentences
from dialoguefactory.policies.base_policies import Policy

class DummyPolicy(Policy):
    """ We use this class to show an example of how to build a policy.
        This policy has no logic implemented, just outputs the following sentence on each dialogue step:
            <player> says: <player> saw <num> unseen sentences.
    """
    def __init__(self, player, dialogue=None):
        self.player = player
        self.dialogue = dialogue
        self.last_seen_cid = 0

    def execute(self, include_goal=False, **params):
        """ Runs the policy. It returns the next agent's response """
        steps = self.get_steps(**params)
        if include_goal:
            return steps, self.get_goal()

        return steps
    
    def get_steps(self, **params):
        """ Returns the next agent's response. Assigns the trusted_source flag to False since
            the DummyPolicy is not a reliable source.
        """
        num_unseen = len(self.dialogue.dia_generator.context)-self.last_seen_cid
        example = sentences.say(self.player,
                                None,
                                "says",
                                sentences.see(self.player, None, "saw", [str(num_unseen), 'unseen', 'sentences'] ),
                                speaker=self.player)
        
        example.trusted_source = False
        self.last_seen_cid = len(self.dialogue.dia_generator.context)
        
        return example
    
    def get_goal(self, **params):
        """ Returns the goal of the policy """
        return None
    
    def save_state(self):
        """ Saves the state that changes over time. This is useful in case you want to save the simulation at some time point.
            For example if you had already evaluated some of the dialogues the state
            of your policy as well. With save and recover state re-evaluation will be possible.
        """
        parent_state = super().save_state()
       
        return (parent_state, self.last_seen_cid)
        
    def recover_state(self, state):
        """ Recovers the saved state """
        parent_state, last_seen_cid = state
        super().recover_state(parent_state)
        self.last_seen_cid = last_seen_cid
    
    


## Evaluating and submitting your solution

We introduced a "hard" environment to test how well the agent generalizes. We connect the "easy" and "hard" environments with locked doors, preserving continuity and making the simulation more realistic. During the test phase, we unlock the doors between the two environments.

In the example below, we evaluate our DummyPolicy on 100 samples. Please evaluate your model and report the metrics on 200.000 dialogues in which the agent participates. We have additional requirements that we explain in the Challenge subsection of our [paper](https://rgdoi.net/10.13140/RG.2.2.17884.19846). An example of evaluation can be seen in the [baseline](baseline.ipynb) notebook.

When generating and evaluating the dialogues, the context can quickly grow and fill up the RAM. For this reason, we have developed an option to flush the dialogue generator. Flushing will save the context string sentences in the "~/dialoguefactory_logs/context.log" file. Flushing the generator does not lose any state because the knowledge base keeps all the information. The parameter that controls the flushing is called flush_after in the generate_and_eval function.


In [ ]:
import dialoguefactory.environments.hard as hard_env
from dialoguefactory.trainers.evaluation import generate_and_eval, pretty_print_eval

In [ ]:
hard_world = hard_env.build_world()
hard_env.merge_worlds(generator, hard_world)

In [ ]:
dummy_policy = DummyPolicy(easy_world.player)
num_test_samples = 100
dias, individual_accuracies, total_accuracy, total_num_dias = generate_and_eval (generator, None, num_test_samples, 
                                                                                 dummy_policy, flush_after=100, notebook_run=True)
    
pretty_print_eval("Example", individual_accuracies, total_accuracy, num_test_samples, 0)

## Debugging

For the same reason as flushing, dialogues by default, are not saved during evaluation by default. So, the *dias* variable in the cell above is empty. To debug the dialogues, please use the parameter *return_dias* set to True in the generate_and_eval function.

If an error occurs during the dialogue run, it is saved in the "~/dialoguefactory_logs/error.log" file.

If you want the dialogues and the generator to remain unchanged after evaluation, set the parameter *forgetful* to True. This is useful if you want to re-evaluate the same dialogues without changing the state of the generator.